In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

import pyarrow
import pyarrow.parquet
from sklearn.cluster import KMeans
from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt


## FUNCTIONS


In [ ]:
def mini_cleaning_pipeline(df: pd.DataFrame) -> pd.DataFrame:

    df = df[df["Type of mobile"].isin(["Class A", "Class B"])].drop(columns=["Type of mobile"])

    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    df = df.drop_duplicates(["Timestamp", "MMSI", ], keep="first")
    df = df.drop(columns=df.columns[-5:])

    knots_to_ms = 0.514444
    df["SOG"] = knots_to_ms * df["SOG"]
    
    return df
    
def quick_summary(df: pd.DataFrame):
    """
    Generate a quick summary of the AIS data in the DataFrame.
    Args:
        df: DataFrame with AIS data, must contain 'MMSI', 'Timestamp', 'Latitude', 'Longitude' columns.
        
    Returns:
        None: Prints summary statistics.
        """
    # Number of unique vessels
    num_vessels = int(df['MMSI'].nunique())
    df = df.rename(columns={"# Timestamp": "Timestamp"})
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

    # Spatial extent
    lat_min, lat_max = df['Latitude'].min(), df['Latitude'].max()
    lon_min, lon_max = df['Longitude'].min(), df['Longitude'].max()
    centroid_lat, centroid_lon = df['Latitude'].mean(), df['Longitude'].mean()

    # Messages per vessel
    msgs_per_vessel = df.groupby('MMSI').size()
    msgs_stats = msgs_per_vessel.describe().to_dict()
    top_10_vessels = msgs_per_vessel.sort_values(ascending=False).head(10)

    # Print concise overview
    print(f"Number of unique vessels: {num_vessels}")
    print(f"Latitude range: {lat_min:.6f} -- {lat_max:.6f}")
    print(f"Longitude range: {lon_min:.6f} -- {lon_max:.6f}")
    print(f"Centroid (lat, lon): ({centroid_lat:.6f}, {centroid_lon:.6f})")
    print("\nMessages per vessel (summary):")
    for k, v in msgs_stats.items():
        print(f"  {k}: {v}")
    print("\nTop 10 vessels by number of messages (MMSI: count):")
    print(top_10_vessels.to_string())


def singularize_vessels(df, mmsi_col="MMSI", start_col_index=11, join_conflicts=True, sep=" | "):
    cols = list(df.columns[start_col_index:])
    if mmsi_col not in df.columns:
        raise KeyError(f"MMSI column '{mmsi_col}' not found in dataframe")

    def _agg(series):
        vals = series.dropna().unique().tolist()
        if len(vals) == 0:
            return np.nan
        if len(vals) == 1:
            return vals[0]
        if join_conflicts:
            return sep.join(map(str, vals))
        return vals

    grouped = df.groupby(mmsi_col)[cols].agg(_agg).reset_index()
    ordered_cols = [mmsi_col] + cols
    return grouped[ordered_cols]

def filter_inside_square(df, bbox) -> pd.DataFrame:
    north, west, south, east = bbox
    df = df[(df["Latitude"] <= north) & (df["Latitude"] >= south) & (df["Longitude"] >= west) & (df["Longitude"] <= east)] 
    return df

def is_inside_polygon(lat, lon, polygon_coords):
    """
    Check if a point (lat, lon) is inside a polygon.
    
    Args:
        lat: Latitude
        lon: Longitude
        polygon_coords: List of (lon, lat) tuples defining polygon vertices
        
    Returns:
        Boolean: True if point is inside polygon
    """
    point = Point(lat, lon)
    polygon = Polygon(polygon_coords)
    return polygon.contains(point)


def extract_static_vessel_info(df, join_conflicts=True, sep=" | "):
    """
    Extract unique vessels with their static information from AIS data.
    Handles conflicting values by joining them or keeping as list.
    
    Args:
        df: Cleaned AIS dataframe with MMSI and static columns
        join_conflicts: If True, join conflicting values with separator. 
                       If False, keep as list.
        sep: Separator for joining conflicting values (default: " | ")
    
    Returns:
        DataFrame with one row per unique MMSI and static columns
    """
    
    # Define static columns
    STATIC_COLUMNS = [
        'MMSI',
        'IMO',
        'Callsign',
        'Name',
        'Ship type',
        'Cargo type',
        'Width',
        'Length',
        'Size A',
        'Size B',
        'Size C',
        'Size D',
    ]
    
    # Filter to only columns that exist in the dataframe
    available_static_cols = [col for col in STATIC_COLUMNS if col in df.columns]
    
    if 'MMSI' not in df.columns:
        raise KeyError("MMSI column not found in dataframe")
    
    # Remove MMSI from aggregation columns (it's the groupby key)
    agg_cols = [col for col in available_static_cols if col != 'MMSI']
    
    def _agg(series):
        """Aggregate function: handle single values, conflicts, or NaN"""
        vals = series.dropna().unique().tolist()
        
        if len(vals) == 0:
            return np.nan
        if len(vals) == 1:
            return vals[0]
        # Multiple conflicting values
        if join_conflicts:
            return sep.join(map(str, vals))
        return vals
    
    # Group by MMSI and aggregate static columns
    static_df = df.groupby('MMSI')[agg_cols].agg(_agg).reset_index()
    
    # Reorder columns to have MMSI first
    ordered_cols = ['MMSI'] + agg_cols
    static_df = static_df[ordered_cols]
    
    # Add derived features
    if 'Size A' in static_df.columns and 'Size B' in static_df.columns:
        static_df['vessel_length'] = pd.to_numeric(static_df['Size A'], errors='coerce') + \
                                     pd.to_numeric(static_df['Size B'], errors='coerce')
    
    if 'Size C' in static_df.columns and 'Size D' in static_df.columns:
        static_df['vessel_width'] = pd.to_numeric(static_df['Size C'], errors='coerce') + \
                                    pd.to_numeric(static_df['Size D'], errors='coerce')
    
    print(f"✅ Extracted {len(static_df)} unique vessels")
    print(f"   Columns: {', '.join(static_df.columns)}")
    
    # Report conflicts
    conflict_cols = []
    for col in agg_cols:
        if static_df[col].astype(str).str.contains(sep, regex=False).any():
            n_conflicts = static_df[col].astype(str).str.contains(sep, regex=False).sum()
            conflict_cols.append(f"{col} ({n_conflicts})")
    
    if conflict_cols:
        print(f"   ⚠️  Conflicts detected in: {', '.join(conflict_cols)}")
    
    return static_df



In [105]:


def df_filter( df: pd.DataFrame, print: bool) -> pd.DataFrame:

    #print initial number of rows and unique vessels
    if print:
        print(f" Starting filtering: {len(df):,} rows, {df['MMSI'].nunique():,} unique vessels")
    #initial filter on bounding box (take northest and southest, westest and eastest points):
    bbox = [57.58, 10.5, 57.12, 11.92]  # north, west, south, east
    # Define polygon coordinates as (lat, lon)
    polygon_coords = [
        (57.3500, 10.5162),  # coast top left
        (57.5120, 10.9314),  # sea top left
        (57.5785, 11.5128),  # sea top right
        (57.5230, 11.9132),  # top right (Swedish coast)
        (57.4078, 11.9189),  # bottom right (Swedish coast)
        (57.1389, 11.2133),  # sea bottom right
        (57.1352, 11.0067),  # sea bottom left
        (57.1880, 10.5400),  # coast bottom left
        (57.3500, 10.5162),  # close polygon (duplicate of first)
    ]

    df = df.rename(columns={"# Timestamp": "Timestamp"}) # Rename column for consistency
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce") # Convert to datetime

    df = df[df["MMSI"].str.len() == 9]  # Adhere to MMSI format
    df = df[df["MMSI"].str[:3].astype(int).between(200, 775)]  # Adhere to MID standard

    df = df.drop_duplicates(["Timestamp", "MMSI", ], keep="first") # Remove duplicates

    #print how many rows and unique vessels are left after filtering
    if print:
        print(f" Initial filtering complete: {len(df):,} rows, {df['MMSI'].nunique():,} unique vessels")

    # Filter based on bounding box
    north, west, south, east = bbox
    df = df[(df["Latitude"] <= north) & (df["Latitude"] >= south) & (df["Longitude"] >= west) & (df["Longitude"] <= east)] 
    if print:
        print(f" Bounding box filtering complete: {len(df):,} rows, {df['MMSI'].nunique():,} unique vessels")
    # Filter based on polygon
    point = df[["Latitude", "Longitude"]].apply(lambda x: Point(x["Longitude"], x["Latitude"]), axis=1)
    polygon = Polygon(polygon_coords)
    df = df[point.apply(lambda x: polygon.contains(x))]
    if print:
        print(f" Polygon filtering complete: {len(df):,} rows, {df['MMSI'].nunique():,} unique vessels")


    knots_to_ms = 0.514444
    df["SOG"] = knots_to_ms * df["SOG"]

    return df

def split_static_dynamic(df, join_conflicts=True, sep=" | "):
    """
    Split AIS dataframe into static vessel info and dynamic trajectory data.
    """
    
    # Define column categories
    STATIC_COLUMNS = [
        'MMSI',
        'IMO',
        'Callsign',
        'Name',
        'Ship type',
        'Cargo type',
        'Width',
        'Length',
        'Size A',
        'Size B',
        'Size C',
        'Size D',
        'Data source type',
        'Type of position fixing device',
    ]
    
    DYNAMIC_COLUMNS = [
        'MMSI',  # Keep as foreign key
        'Timestamp',
        'Type of mobile',
        'Latitude',
        'Longitude',
        'Navigational status',
        'ROT',
        'SOG',
        'COG',
        'Heading',
        'Draught',
        'Destination',
        'ETA',
    ]
    
    if 'MMSI' not in df.columns:
        raise KeyError("MMSI column not found in dataframe")
    
    # 1. CREATE STATIC DATAFRAME
    available_static = [col for col in STATIC_COLUMNS if col in df.columns]
    agg_cols = [col for col in available_static if col != 'MMSI']
    
    def _agg(series):
        vals = series.dropna().unique().tolist()
        if len(vals) == 0:
            return np.nan
        if len(vals) == 1:
            return vals[0]
        if join_conflicts:
            if "Unknown" in vals:
                vals.remove("Unknown")
            if "Undefined" in vals:
                vals.remove("Undefined")
            if len(vals) == 1:
                return vals[0]
            return sep.join(map(str, vals))
        return vals
    
    static_df = df.groupby('MMSI')[agg_cols].agg(_agg).reset_index()
    
    
    # 2. CREATE DYNAMIC DATAFRAME
    available_dynamic = [col for col in DYNAMIC_COLUMNS if col in df.columns]
    dynamic_df = df[available_dynamic].copy()
    
    # 3. REPORT
    print(f"Split complete:")
    print(f"   Static:  {len(static_df):,} unique vessels with {len(static_df.columns)} columns")
    print(f"   Dynamic: {len(dynamic_df):,} AIS messages with {len(dynamic_df.columns)} columns")
    
    # Check for conflicts in static data
    conflict_cols = []
    for col in agg_cols:
        if static_df[col].astype(str).str.contains(sep, regex=False).any():
            n_conflicts = static_df[col].astype(str).str.contains(sep, regex=False).sum()
            conflict_cols.append(f"{col} ({n_conflicts})")
    
    if conflict_cols:
        print(f"  Static conflicts: {', '.join(conflict_cols)}")
    
    return static_df, dynamic_df


# 🛰️ AIS Data Filtering and Preprocessing — Kattegat Submarine Cable Area

This notebook documents the initial steps of data preparation for the AIS anomaly detection project focusing on the submarine communication cables in the Kattegat area (Denmark–Sweden).

The goal is to isolate AIS data within a defined polygon surrounding three specific cable routes (**GC2**, **Kattegat 2A**, and **Kattegat 2B**) and perform preliminary cleaning and filtering.


## 🧭 Area of Interest Definition

We are interested in identifying potential anomalous or risky vessel behaviors near submarine cables.

By Danish law, a 200 m protection zone exists on each side of these cables, but in this study we extend the inspection zone to **5 km** to capture behavioral precursors such as early anchoring, trawling, or route deviations before ships enter the restricted corridor.

The polygon defining our area of interest was manually approximated based on the **DKCPC map**.

It includes a section of the Danish coast near Saeby and extends to the Swedish side around Lerkil, covering the main cable routes.  
The coordinates are stored as `(latitude, longitude)` pairs and define an octagonal region enclosing the study zone.


In [ ]:
#initial filter on bounding box (take northest and southest, westest and eastest points):
bbox = [57.58, 10.5, 57.12, 11.92]  # north, west, south, east
# Define polygon coordinates as (lat, lon)
polygon_coords = [
    (57.3500, 10.5162),  # coast top left
    (57.5120, 10.9314),  # sea top left
    (57.5785, 11.5128),  # sea top right
    (57.5230, 11.9132),  # top right (Swedish coast)
    (57.4078, 11.9189),  # bottom right (Swedish coast)
    (57.1389, 11.2133),  # sea bottom right
    (57.1352, 11.0067),  # sea bottom left
    (57.1880, 10.5400),  # coast bottom left
    (57.3500, 10.5162),  # close polygon (duplicate of first)
]


## ⚙️ Data Loading

In this step, AIS data (in CSV format) is loaded and receives a first hand cleaning 10% of the data

In [ ]:
df_28 = pd.read_csv("aisdk-2025-10-28.csv")

In [ ]:
#check how many rows are in the dataframe
df_28.shape

In [ ]:
df_cleaned = mini_cleaning_pipeline(df_28)

In [ ]:
df_cleaned.shape

## 📍 SQUARE Filtering

The box coordinates defined earlier are used to filter the AIS dataset.
Each point is checked to determine whether it falls inside the box using simple operations

In [ ]:
df_inside_square = filter_inside_square(df_cleaned, bbox)

In [ ]:
df_inside_square.shape

## 📍 Polygon Filtering

The polygon coordinates defined earlier are used to filter the AIS dataset.

Each point is checked to determine whether it falls inside the polygon using geometric operations (e.g. with the `shapely` library).

The resulting dataset contains only positions **within the defined Kattegat zone**, roughly **3–4 %** of the original AIS dataset.


In [ ]:
df_katt_2 = df_inside_square[
    df_inside_square.apply(lambda row: is_inside_polygon(row['Latitude'], row['Longitude'], polygon_coords), axis=1)
]

In [ ]:
df_katt_2.shape

In [ ]:
#here import directly the cleaned one from csv
df_katt = pd.read_csv("df_28_katt.csv")

## 📊 Data Overview After Filtering

We summarize the filtered dataset:

- Number of unique vessels  
- Time coverage of the subset  
- Spatial extent and approximate number of messages per vessel  

This quick overview helps confirm that the filtering worked as expected and that the dataset is representative of the study area.


In [ ]:
quick_summary(df_katt)

# USING SUMMARY FUNCTIONS TO FILTER AND DIVIDE

In [106]:
df_filtered = df_filter(df_28, pr=True)

TypeError: df_filter() got an unexpected keyword argument 'pr'

In [101]:
df_static, df_dynamic = split_static_dynamic(df_filtered, join_conflicts=True, sep=" | ")

✅ Split complete:
   Static:  173 unique vessels with 10 columns
   Dynamic: 322,988 AIS messages with 13 columns
   ⚠️  Static conflicts: Type of position fixing device (1)


In [102]:
df_static.head()

,MMSI,IMO,Callsign,Name,Ship type,Cargo type,Width,Length,Data source type,Type of position fixing device
0,2190073,Unknown,Unknown,NaN,Undefined,NaN,NaN,NaN,AIS,GPS
1,111219510,Unknown,0,NaN,SAR,NaN,5.0,20.0,AIS,GPS
2,111265121,Unknown,Unknown,FFK STC 121,Reserved,NaN,10.0,8.0,AIS,Undefined
3,209114000,9173185,5BQN4,RIX AMETHYST,Cargo,NaN,12.0,89.0,AIS,Internal
4,209903000,9975753,5BHK6,CEMCOMMANDER,Cargo,No additional information,15.0,113.0,AIS,GPS
